In [0]:
# Imports 

%tensorflow_version 2.x
import tensorflow as tf
from tensorflow.python.keras.applications.vgg16 import VGG16
from tensorflow.python.keras.applications.inception_v3 import InceptionV3
from matplotlib import pyplot
from tensorflow.python import keras
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Dense,Conv2D, MaxPooling2D ,Activation, Dropout, Flatten, BatchNormalization, GlobalAveragePooling2D
from tensorflow.python.keras import Sequential
from tensorflow.python.keras.regularizers import l2
from tensorflow.python.keras.optimizers import SGD
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator,load_img 

In [0]:
# Model Configuration
img_width, img_height = 224, 224  
train_data_dir = './ProgDataRandom/prog_data/training/'
validation_data_dir = './ProgDataRandom/prog_data/validation/'
test_data_dir = './ProgDataRandom/prog_data/testing/'
nb_train_samples = 8500*5
nb_validation_samples = 1000*5
epochs = 50
batch_size = 50
reg=0.0001
learning_rate = 0.01

In [0]:
# Setting up the image format/image shape
if K.image_data_format() == 'channels_first': 
    input_shape = (3, img_width, img_height) 
else: 
    input_shape = (img_width, img_height, 3)
 

In [7]:
# Created a model by using the "InceptionV3" as basemodel 
# and also appended few more layers to the base model

base_model = InceptionV3(weights ="imagenet",include_top= False,input_shape=input_shape )

x = base_model.output
x = GlobalAveragePooling2D()(x)

x = Dense(512,kernel_initializer='he_normal',kernel_regularizer=l2(reg),bias_regularizer=l2(reg), activation='relu')(x)
x = Dense(256,kernel_initializer='he_normal',kernel_regularizer=l2(reg),bias_regularizer=l2(reg), activation='relu')(x)
x = Dense(128,kernel_initializer='he_normal',kernel_regularizer=l2(reg),bias_regularizer=l2(reg), activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(5, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = True

model.summary()

sgd = tf.keras.optimizers.SGD(lr=learning_rate, decay=1e-6, momentum=0.9, nesterov=True)

model.compile( loss= keras.losses.categorical_crossentropy, 
               optimizer = sgd, 
               metrics=['accuracy'])

87916544/87910968 [==============================] - 2s 0us/step
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
_____________________________

In [8]:
# The section is responsible for feeding the data during the training process
datagen = ImageDataGenerator( rescale = 1./255,
                              rotation_range=20,
                              zoom_range=0.15,
                              width_shift_range=0.2,
                              height_shift_range=0.2,
                              shear_range=0.15,
                              horizontal_flip=True,
                              fill_mode="nearest")   
test = ImageDataGenerator(rescale=1./255)
  
train_generator = datagen.flow_from_directory(
                train_data_dir, 
                target_size =(img_width, img_height), 
                batch_size = batch_size,
                class_mode ='categorical',
                classes=['class_0','class_1','class_2','class_3','class_4'],
                shuffle = True
                ) 

validation_generator = datagen.flow_from_directory( 
                validation_data_dir, 
                target_size =(img_width, img_height), 
                batch_size = batch_size,
                class_mode ='categorical',
                classes=['class_0','class_1','class_2','class_3','class_4'],
                shuffle = True
                )
test_generator = test.flow_from_directory( 
                test_data_dir, 
                target_size =(img_width, img_height), 
                batch_size = batch_size,
                class_mode ='categorical',
                classes=['class_0','class_1','class_2','class_3','class_4'],
                shuffle = True
                )


Found 42500 images belonging to 5 classes.
Found 5000 images belonging to 5 classes.
Found 2500 images belonging to 5 classes.


In [ ]:
# Started the Training process

from tensorflow.python.keras.callbacks import ModelCheckpoint
filepath="weights_best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
history= model.fit(
    train_generator, 
    steps_per_epoch = nb_train_samples // batch_size, 
    epochs = epochs,
    validation_data = validation_generator, 
    validation_steps = nb_validation_samples // batch_size,
    shuffle = True,callbacks=callbacks_list
    )


model.save_weights('./model_saved_full.h5',overwrite= True)

In [0]:
# The best trainned weights are stored in my GDrive as I was using colab for trainning purpose
# So, loading the best weight from my GDrive

model.load_weights("/content/drive/My Drive/Diabetic Retinopathy InceptionV3 weights/InceptionV3_best_7796.hdf5")

In [10]:
# performing Evaluation

print('\n# Evaluate on test data')
results = model.evaluate(test_generator,batch_size = batch_size,steps = 2500 // batch_size, use_multiprocessing=True)
print('test loss, test acc:', results)


# Evaluate on test data
50/50 [==============================] - 8s 151ms/step - loss: 0.9731 - accuracy: 0.7280
test loss, test acc: [0.9730538129806519, 0.7279999852180481]


In [11]:
# Testing the model with the best weight loaded 
# Examines the Testing set/ makes prediction of the testing set

import numpy as np
import os
import cv2
from sklearn.metrics import classification_report, confusion_matrix
#Y_pred = model.predict(test_generator,batch_size = batch_size,steps = 2500 // batch_size, use_multiprocessing=True)
label=[]
clas=[]
test_data_dir = './ProgDataRandom/prog_data/testing/'
for j,i in enumerate(np.sort(os.listdir(test_data_dir))):
    p=os.path.join(test_data_dir,i)
    for image in os.listdir(p):

            inp = os.path.join(p,image)
            img = np.asarray(load_img(inp))/255
            img = np.reshape(img,[1,img_width,img_height,3])

            classes = np.argmax(model.predict(img))
            
            clas.append(classes)
            label.append(j)
    print(j," completed.")
          


0  completed.
1  completed.
2  completed.
3  completed.
4  completed.


In [0]:
print(label,clas,sep="\n")

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [12]:
# Generates the Confusion Matrix and other Reports
print('Confusion Matrix')
print(confusion_matrix(label, clas))
print('Classification Report')
target_names = ['class_0','class_1','class_2','class_3','class_4']
print(classification_report(label, clas, target_names=target_names))

Confusion Matrix
[[416  42  34   7   1]
 [107 383  10   0   0]
 [124  72 232  69   3]
 [ 74  11 122 293   0]
 [  1   2   1   0 496]]
Classification Report
              precision    recall  f1-score   support

     class_0       0.58      0.83      0.68       500
     class_1       0.75      0.77      0.76       500
     class_2       0.58      0.46      0.52       500
     class_3       0.79      0.59      0.67       500
     class_4       0.99      0.99      0.99       500

    accuracy                           0.73      2500
   macro avg       0.74      0.73      0.72      2500
weighted avg       0.74      0.73      0.72      2500



In [ ]:
# Use to visualize the result graphically

f1=pyplot.figure()
pyplot.plot(history.history['accuracy'], label='train_acc')
pyplot.plot(history.history['val_accuracy'], label='val_acc')
f2=pyplot.figure()
pyplot.plot(history.history['loss'], label='train_loss')
pyplot.plot(history.history['val_loss'], label='val_loss')
pyplot.legend()
pyplot.show() 